In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/job-dataset/unlabelled_development_set.csv
/kaggle/input/job-dataset/no_agument_training_5118.json
/kaggle/input/job-dataset/prompts_complete_test_salary.json
/kaggle/input/job-dataset/work_arrangements_development_set.csv
/kaggle/input/job-dataset/prompts_complete_test_work.json
/kaggle/input/job-dataset/work_arrangements_test_set.csv
/kaggle/input/job-dataset/combined_prompts_complete_6k.json
/kaggle/input/job-dataset/seniority_labelled_test_set.csv
/kaggle/input/job-dataset/agument_training_7800.json
/kaggle/input/job-dataset/seniority_labelled_development_set.csv
/kaggle/input/job-dataset/prompts_complete_test_seniority.json
/kaggle/input/job-dataset/salary_labelled_development_set.csv
/kaggle/input/job-dataset/test_1355.json
/kaggle/input/job-dataset/combined_prompts_complete_train_80.json
/kaggle/input/job-dataset/combined_prompts_complete_val_20.json
/kaggle/input/job-dataset/salary_labelled_test_set.csv


**Note - Setting of this notebook model**

LLM model: qwen0.5b

Training method: prompt tuning

epoch:4

batch:8

learning rate: 3e-3

training set: /kaggle/input/job-dataset/agument_training_7800.json

test set: /kaggle/input/job-dataset/test_1355.json


In [ ]:
input_path = "/kaggle/input/job-dataset/agument_training_7800.json"
input_df = pd.read_json(input_path)
input_df

,prompt,complete
0,[TASK: Salary] Extract salary information from...,17500-17500-PHP-MONTHLY
1,[TASK: Salary] Extract salary information from...,16000-16000-PHP-MONTHLY
2,[TASK: Salary] Extract salary information from...,0-0-None-None
3,[TASK: Salary] Extract salary information from...,0-0-None-None
4,[TASK: Salary] Extract salary information from...,0-0-None-None
...,...,...
7795,[TASK: Work Arrangement] Extract work arrangem...,Hybrid
7796,[TASK: Work Arrangement] Extract work arrangem...,OnSite
7797,[TASK: Work Arrangement] Extract work arrangem...,Remote
7798,[TASK: Work Arrangement] Extract work arrangem...,Hybrid


In [ ]:
import os
import time
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model,
    PromptTuningConfig,
    TaskType
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


model_path = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to('cpu')

2025-04-24 12:44:41.195806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745498681.400305      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745498681.459930      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Extract key information from the text:",
    num_virtual_tokens=20,
    tokenizer_name_or_path='Qwen/Qwen2.5-0.5B-Instruct',
    token_dim=model.config.hidden_size,
    num_transformer_submodules=1
)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = get_peft_model(model, peft_config)

# 3. Move the *PEFT model* to GPU
model = model.to(device)


# Assume there is no cuda (if there is cuda, uncomment the above code instead)
# model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 17,920 || all params: 494,050,688 || trainable%: 0.0036


In [ ]:

def load_and_preprocess(data_path):
    dataset = load_dataset("json", data_files=data_path)

    dataset = dataset.filter(lambda x: len(x["prompt"]) > 0 and len(x["complete"]) > 0)

    def tokenize_function(examples):

        completions = [str(c) for c in examples["complete"]]

        texts = [p + c + tokenizer.eos_token for p, c in zip(examples["prompt"], completions)]

        tokenized = tokenizer(
            texts,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        prompt_tokenized = tokenizer(examples["prompt"], add_special_tokens=False)
        prompt_lengths = [len(ids) for ids in prompt_tokenized["input_ids"]]

        labels = tokenized["input_ids"].clone()
        for i, length in enumerate(prompt_lengths):
            if length >= 512:
                length = 511
            labels[i][:length] = -100

        tokenized["labels"] = labels
        return tokenized

    return dataset.map(tokenize_function, batched=True)


In [ ]:
from torch.cuda import max_memory_allocated, empty_cache

In [ ]:
def formal_train(json_path="salary_47_allin.json", target_model_path="./qwen_prompt_47"):
    dataset = load_and_preprocess(json_path)
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./qwen_prompt_tuning_output",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs=4, # change this from 4 to 6
        learning_rate=3e-3,
        logging_dir="./logs",
        logging_steps=100,
        save_strategy="epoch",
        fp16=True,
        optim="adamw_torch",
        dataloader_num_workers=4,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        data_collator=data_collator,
    )

    print("开始训练...")
    start_time = time.time()
    trainer.train()
    print(f"训练完成，耗时: {time.time() - start_time:.2f}秒")

    model.save_pretrained(target_model_path)

    # To check memory useage
    # Optional: free unused memory before checking
    torch.cuda.empty_cache()
    # Memory usage check (in GB)
    print(f"Peak memory usage: {max_memory_allocated() / 1024**3:.2f} GB")


In [ ]:
formal_train(json_path=input_path, target_model_path="./qwen_05b_4epochs_prompt_fix_8batch_augdata")

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/7800 [00:00<?, ? examples/s]

Map:   0%|          | 0/7800 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


开始训练...


Step,Training Loss
100,1.501100
200,1.440900
300,1.438900
400,1.415100
500,1.417600
600,1.423900
700,1.428800
800,1.410900
900,1.415200


训练完成，耗时: 1646.93秒
Peak memory usage: 12.47 GB


In [ ]:

def generate_response(prompt):
  with torch.no_grad():
      inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
      outputs = model.generate(
          input_ids=inputs["input_ids"],
          attention_mask=inputs["attention_mask"],
          max_new_tokens=100,
          temperature=0.7,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )

      generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
      return tokenizer.decode(generated_tokens, skip_special_tokens=True)


In [ ]:
test_path = "/kaggle/input/job-dataset/test_1355.json"
test_df = pd.read_json(test_path)
test_df

,prompt,complete
0,[TASK: Salary] Extract salary information from...,1500-1800-MYR-MONTHLY
1,[TASK: Salary] Extract salary information from...,60-60-HKD-HOURLY
2,[TASK: Salary] Extract salary information from...,0-0-None-None
3,[TASK: Salary] Extract salary information from...,0-0-None-None
4,[TASK: Salary] Extract salary information from...,0-0-None-None
...,...,...
1350,[TASK: Work Arrangement] Extract work arrangem...,OnSite
1351,[TASK: Work Arrangement] Extract work arrangem...,Hybrid
1352,[TASK: Work Arrangement] Extract work arrangem...,OnSite
1353,[TASK: Work Arrangement] Extract work arrangem...,OnSite


In [ ]:
import json
with open(test_path, 'r') as f: #Update with the path to your JSON File.
    data = json.load(f)

In [ ]:
len(data)

1355

In [ ]:
# Check the id of each prompt task
# Your task-specific prompt instructions
prompt_dict = {
    'salary': '[TASK: Salary]',
    'seniority': '[TASK: Seniority]',
    'arrangement': '[TASK: Work Arrangement]'
}

# Keep track of first occurrence indices
task_start_indices = {}

# Loop through val set to find the first index of each task
for i, item in enumerate(data):
    for task_name, prompt in prompt_dict.items():
        # task is 'salary','seniority','arrangement
        # prompt is 'Extract salary information from the job ad. \n\n' ...
        if prompt in item['prompt']:
            if task_name not in task_start_indices:
                task_start_indices[task_name] = i
        if len(task_start_indices) == 3:
          break
print("First index for each task:")
print(task_start_indices)

First index for each task:
{'salary': 0, 'seniority': 567, 'arrangement': 1256}


In [ ]:
salary_data = data[:567]
seniority_data = data[567:1256]
work_data = data[1256:]

In [ ]:
salary_data[:3]

[{'prompt': '[TASK: Salary] Extract salary information from the job ad. \n\njob ad: \nCashier (Kota Tinggi) Bertanggungjawab sebagai cashier\nMengurus semua rekod mengenai cek yang diterima\nMenyediakan laporan yang diperlukan oleh HQ (Jabatan Akaun dan Jabatan Sumber Manusia)\nKiraan stok bulanan\nSemua kerja lain yang ditetapkan oleh pengurus cawangan dan supervisor pada bila-bila mengikut keperluan\nKeperluan\nBerkelulusan SPM / O Level / SKM Level 1 / SKM Level 2 / SKM Level 3 atau setaraf\nSedikit kemahiran tentang komputer\nMenepati masa\nKerja overtime (Jika diperlukan)\nGaji RM 1500 – 1800++ Calon berminat boleh whatsapp 010-3938581\nSeng Li Marketing Sdn Bhd is a One-Stop Auto Parts Trading Company\nSalary :\nRM 1500 – 1800 MY RM\xa01,500 – RM\xa01,800 per month\n\n',
  'complete': '1500-1800-MYR-MONTHLY'},
 {'prompt': '[TASK: Salary] Extract salary information from the job ad. \n\njob ad: \n學校派飯員 編號 :\n35-24-0004254 LA 兼職\n日期 :\n01/02/2024\n職位 :\n學校派飯員\n公司/僱主名稱 :\n栢俐食品集團有限公司\

In [ ]:
seniority_data[:3]

[{'prompt': '[TASK: Seniority] Extract seniority level from the job ad. \n\njob ad: \nFinancial Planning and Modelling Manager . Help us create better connected futures.\nMake your mark. See the difference. Be recognised.\nSolve high-impact, complex challenges\nWork with aligned engaged team members\nAbout us\nAt BAI, we draw from our rich history to create even better connected futures.\nWe are both big enough to deliver and maintain large-scale operations and nimble enough to make things happen.\nWe deliver Wi-Fi and cellular connectivity on subways from New York to Hong Kong to Toronto and we keep Australians, from the cities to the outback, connected to what matters to them.\nWe build technology, and teams, that people want to be part of, and we have the courage to not only imagine, but to do, what really matters.\nHelp us imagine and create what’s next. Join us.\nAbout the role\nPlay a key role in providing support to the Group Planning & Reporting Manager to set up a ‘Centre of E

In [ ]:
work_data[:3]

[{'prompt': '[TASK: Work Arrangement] Extract work arrangement from the job ad. \n\njob ad: \nJob title: Restaurant Kitchen Hand\nAbstract: We are seeking experienced Kitchen Hand to join our hospitality team.\nEmployer: Catering HQ\nLocation: pitttown\nHighlights: opportunity for growth, Opportunity to work in an industry leading hospitality group, Positive, fun and supportive work culture\nContents: We are currently searching for talented and polished Full Time Kitchen Hands to join our hospitality team.\nKey duties\nThorough cleaning of the kitchen, including dishes and floors, Food preparation assistance, Stock rotation and stock control, Ensuring to follow all health and safety procedures when caring out all tasks, Any other adhoc duties as required by our fantastic Chefs and Management, Operating a commercial dishwasher to a high standard, Assist with general kitchen duties under the direction of the Head Chef.\nThe Person\nPrevious relevant experience in a high-volume catering/k

In [ ]:
# Inference test_data
# Limit to only first 10 examples for each task to preview
for i, item in enumerate(data):
  if (0 <= i < 10) or (567 <= i < 577) or (1345 <= i):
    print("=" * 20, 'Round:', i, "=" * 20)
    p = item['prompt']
    y = item['complete']
    print("Target:", y)
    print("\n Prompt tuning Answer:", generate_response(p))

==================== Round: 0 ====================
Target: 1500-1800-MYR-MONTHLY


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:1889: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")



 Prompt tuning Answer: 1500-1800-MYR-MONTHLY
==================== Round: 1 ====================
Target: 60-60-HKD-HOURLY

 Prompt tuning Answer: 60-60-000000-$000-HR-MONTHLY
==================== Round: 2 ====================
Target: 0-0-None-None

 Prompt tuning Answer: 0-0-None-None
==================== Round: 3 ====================
Target: 0-0-None-None

 Prompt tuning Answer: 0-0-None-None
==================== Round: 4 ====================
Target: 0-0-None-None

 Prompt tuning Answer: 0-0-None-None
==================== Round: 5 ====================
Target: 21-21-NZD-HOURLY

 Prompt tuning Answer: 21-21-NZD-MONTHLY
==================== Round: 6 ====================
Target: 0-0-None-None

 Prompt tuning Answer: 0-0-None-None
==================== Round: 7 ====================
Target: 0-0-None-None

 Prompt tuning Answer: 0-0-None-None
==================== Round: 8 ====================
Target: 32-32-AUD-HOURLY

 Prompt tuning Answer: 32-32-None-None
==================== Round: 9 ======

# **Evaluation**

In [ ]:
# Evaluation on Test Set
device = "cuda" if torch.cuda.is_available() else "cpu"
mode = model.to(device)
model.eval()

# For Inference
def generate_response(prompt):
  with torch.no_grad():
      inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
      outputs = model.generate(
          input_ids=inputs["input_ids"],
          attention_mask=inputs["attention_mask"],
          max_new_tokens=100,
          temperature=0.7,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )
      generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
      return tokenizer.decode(generated_tokens, skip_special_tokens=True)

# Salary Evaluation

# # Test data path
# test_data_path = "/kaggle/input/job-dataset/prompts_complete_test_salary.json"

# # Load test data
# with open(test_data_path, 'r') as f: #Update with the path to your JSON File.
#     test_data = json.load(f)


# Exact match Evaluation
def exact_match_accuracy(test_data):
    true_count = 0
    total_count = 0
    for i, item in enumerate(test_data):
        total_count += 1
        p = item['prompt']
        y_true = item['complete']
        y_pred = generate_response(p)
        if y_true == y_pred:
            true_count += 1
    print(f"Salary Exact Match Accuracy: {((true_count/total_count) * 100):.2f}%\n")

# Separate Match Evaluation
def separate_match_accuracy(test_data):
    # Initialization
    total_count = 0
    true_min_count = 0
    true_max_count = 0
    true_currency_count = 0
    true_frequency_count = 0
    y_min_sal_pred = -999999 # default wrong answer
    y_max_sal_pred = -999999 # default wrong answer
    y_currency_pred = -999999 # default wrong answer
    y_frequency_pred = -999999 # default wrong answer

    for i, item in enumerate(test_data):
        total_count += 1
        p = item['prompt']
        y_true = item['complete']
        y_pred = generate_response(p)

        # Separate y_true to 4 parts
        y_true_list = y_true.split('-')
        y_min_sal_true = y_true_list[0]
        y_max_sal_true = y_true_list[1]
        y_currency_true = y_true_list[2]
        y_frequency_true = y_true_list[3]

        if '-' in y_pred:
            y_pred_list = y_pred.split('-')

            # Check the number of parts
            num_parts = len(y_pred_list)

            # We consider the oder of the parts matter in evaluation
            if num_parts == 4:
                y_min_sal_pred = y_pred_list[0]
                y_max_sal_pred = y_pred_list[1]
                y_currency_pred = y_pred_list[2]
                y_frequency_pred = y_pred_list[3]
            elif num_parts == 3:
                y_min_sal_pred = y_pred_list[0]
                y_max_sal_pred = y_pred_list[1]
                y_currency_pred = y_pred_list[2]
                y_frequency_pred = -999999 # default wrong answer
            elif num_parts == 2:
                y_min_sal_pred = y_pred_list[0]
                y_max_sal_pred = y_pred_list[1]
                y_currency_pred = -999999 # default wrong answer
                y_frequency_pred = -999999 # default wrong answer
            elif num_parts == 1:
                y_min_sal_pred = y_pred_list[0]
                y_max_sal_pred = -999999 # default wrong answer
                y_currency_pred = -999999 # default wrong answer
                y_frequency_pred = -999999 # default wrong answer
        else:
            # There is no "-" in y_pred
            # We assume all answers are wrong in this case
            y_min_sal_pred = -999999 # default wrong answer
            y_max_sal_pred = -999999 # default wrong answer
            y_currency_pred = -999999 # default wrong answer
            y_frequency_pred = -999999 # default wrong answer

        # Compare results
        if y_min_sal_true == y_min_sal_pred:
            true_min_count += 1
        if y_max_sal_true == y_max_sal_pred:
            true_max_count += 1
        if y_currency_true == y_currency_pred:
            true_currency_count += 1
        if y_frequency_true == y_frequency_pred:
            true_frequency_count += 1
    print(f"Overall Salary Average Accuracy: { ((((true_min_count/total_count) + (true_max_count/total_count) + (true_currency_count/total_count) + (true_frequency_count/total_count))/4) * 100):.2f}%")
    print(f"Min Salary Accuracy: {((true_min_count/total_count) * 100):.2f}%")
    print(f"Max Salary Accuracy: {((true_max_count/total_count) * 100):.2f}%")
    print(f"Currency Accuracy: {((true_currency_count/total_count) * 100):.2f}%")
    print(f"Payment Frequency Accuracy: {((true_frequency_count/total_count) * 100):.2f}% \n")

        # print(f"y_min_sal_pred: {y_min_sal_pred}")
        # print(f"y_max_sal_pred: {y_max_sal_pred}")
        # print(f"y_currency_pred: {y_currency_pred}")
        # print(f"y_frequency_pred: {y_frequency_pred}")

# To evaluate separate match performance against test set
print(f"Separte Match Evaluation Begins")
start_time = time.time()
separate_match_accuracy(salary_data)
print(f"Separte Match Evaluation time: {(time.time() - start_time):.2f} s")
print()

# To evaluate exact match performance against test set
print(f"Exact Match Evaluation Begins")
start_time = time.time()
exact_match_accuracy(salary_data)
print(f"Exact Match Evaluation time: {(time.time() - start_time):.2f} s")

Separte Match Evaluation Begins
Overall Salary Average Accuracy: 65.26%
Min Salary Accuracy: 78.84%
Max Salary Accuracy: 65.08%
Currency Accuracy: 64.02%
Payment Frequency Accuracy: 53.09% 

Separte Match Evaluation time: 220.22 s

Exact Match Evaluation Begins
Salary Exact Match Accuracy: 43.56%

Exact Match Evaluation time: 228.19 s


In [ ]:
# Seniority Evaluation

# # Test data path
# test_data_path = "/kaggle/input/job-dataset/prompts_complete_test_seniority.json"

# # Load test data
# with open(test_data_path, 'r') as f: #Update with the path to your JSON File.
#     test_data = json.load(f)

# Exact match Evaluation
def exact_match_accuracy(test_data):
    true_count = 0
    total_count = 0
    for i, item in enumerate(test_data):
        total_count += 1
        p = item['prompt']
        y_true = item['complete']
        y_pred = generate_response(p)
        if y_true == y_pred:
            true_count += 1
    print(f"Seniority Exact Match Accuracy: {((true_count/total_count) * 100):.2f}% \n")

# To evaluate exact match performance against test set
print(f"Seniority Evaluation Begins")
start_time = time.time()
exact_match_accuracy(seniority_data)
print(f"Seniority Evaluation time: {(time.time() - start_time):.2f} s")


Seniority Evaluation Begins
Seniority Exact Match Accuracy: 35.56% 

Seniority Evaluation time: 61.92 s


In [ ]:
# Work Evaluation

# # Test data path
# test_data_path = "/kaggle/input/job-dataset/prompts_complete_test_work.json"

# # Load test data
# with open(test_data_path, 'r') as f: #Update with the path to your JSON File.
#     test_data = json.load(f)


# Exact match Evaluation
def exact_match_accuracy(test_data):
    true_count = 0
    total_count = 0
    for i, item in enumerate(test_data):
        total_count += 1
        p = item['prompt']
        y_true = item['complete']
        y_pred = generate_response(p)
        if y_true == y_pred:
            true_count += 1
    print(f"Work Exact Match Accuracy: {((true_count/total_count) * 100):.2f}% \n")

# To evaluate exact match performance against test set
print(f"Work Evaluation Begins")
start_time = time.time()
exact_match_accuracy(work_data)
print(f"Work Evaluation time: {(time.time() - start_time):.2f} s")


Work Evaluation Begins
Work Exact Match Accuracy: 0.00% 

Work Evaluation time: 57.93 s
